In [3]:
import requests
import csv

api_key = "1WD02A0ZXUENB863" #Alpha Vantage

from_currency = "USD"
to_currency = "INR"
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={from_currency}&to_symbol={to_currency}&apikey={api_key}&outputsize=full"

# Make the API request
response = requests.get(url)
data = response.json()

# Extract historical data
time_series = data.get("Time Series FX (Daily)", {})

# Define the CSV file name
csv_file = "currency_exchange_rate.csv"

# Save data to a CSV file
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Exchange Rate (USD/INR)"])
    for date, daily_data in time_series.items():
        exchange_rate = daily_data.get("4. close", "")
        writer.writerow([date, exchange_rate])

print(f"Historical exchange rate data saved to {csv_file}")


Historical exchange rate data saved to currency_exchange_rate.csv


In [5]:
import pandas as pd

# Mapping quarters to months
quarter_to_month = {
    "Q1": "04",  
    "Q2": "07",  
    "Q3": "10",  
    "Q4": "01"   # next year
}

file_path = "gdp_growth_rates.csv" 
df = pd.read_csv(file_path)

def convert_to_date(fy_quarter):
    fy, quarter = fy_quarter.split(" (")
    fy_start = int(fy.split("-")[0])
    month = quarter_to_month[quarter.strip(")")]

    year = fy_start if month != "01" else fy_start + 1
    return f"{year}-{month}-01"  

df["Date"] = df["Year"].apply(convert_to_date)

df.drop(columns=["Year"], inplace=True)

df.to_csv(file_path, index=False)

print("Date conversion completed and saved to the same CSV file.")


Date conversion completed and saved to the same CSV file.


In [12]:
#merge usa and india gdp growth rated into one file 
import pandas as pd

india_file = "datasets/gdp_growth_rates_india.csv"
usa_file = "datasets/gdp_growth_rates_usa.csv"
merged_file = "gdp_growth_rates.csv"

india_df = pd.read_csv(india_file)
usa_df = pd.read_csv(usa_file)

india_df["Date"] = pd.to_datetime(india_df["Date"], format="%d-%m-%Y")
usa_df["Date"] = pd.to_datetime(usa_df["observation_date"], dayfirst=True)  # Set dayfirst=True for DD-MM-YYYY format

india_df.rename(columns={"Real GDP Growth Rate India": "GDP Rate India"}, inplace=True)
usa_df.rename(columns={"A191RL1Q225SBEA": "GDP Rate USA"}, inplace=True)

merged_df = pd.merge(india_df[["Date", "GDP Rate India"]], usa_df[["Date", "GDP Rate USA"]], on="Date", how="outer")

merged_df.sort_values(by="Date", inplace=True)

merged_df.to_csv(merged_file, index=False)

print(f"Merged data saved to {merged_file}")


Merged data saved to gdp_growth_rates.csv


In [15]:
#merge usa and india interest rates into one file
import pandas as pd

usa_file = "datasets/interest_rates_usa.csv"
india_file = "datasets/interest_rates_india.csv"
merged_file = "interest_rates.csv"

usa_df = pd.read_csv(usa_file)
india_df = pd.read_csv(india_file)

usa_df["observation_date"] = pd.to_datetime(usa_df["observation_date"], format="%Y-%m-%d")
india_df["observation_date"] = pd.to_datetime(india_df["observation_date"], format="%Y-%m-%d")

merged_df = pd.merge(usa_df, india_df, on="observation_date", how="inner")

merged_df.rename(
    columns={
        "FEDFUNDS": "Interest Rate USA",
        "IRSTCI01INM156N": "Interest Rate India"
    },
    inplace=True,
)

merged_df.to_csv(merged_file, index=False)

print(f"Merged data has been saved to {merged_file}.")


Merged data has been saved to interest_rates.csv.


In [20]:
#converting into standard date format
import pandas as pd

file_path = "trade_balance.csv"
df = pd.read_csv(file_path)

df['Month'] = pd.to_datetime(df['Month'], format='%b-%y').dt.strftime('%Y-%m-01')

df.to_csv(file_path, index=False)

print("Date format updated successfully and saved to the same file.")

Date format updated successfully and saved to the same file.


In [12]:
# from nsetools import Nse
# import pandas as pd
# from datetime import datetime

# nse = Nse()

# def fetch_nifty_50_live():
#     try:
#         index_data = nse.get_index_quote("nifty 50")
#         current_date = datetime.now().strftime("%Y-%m-%d")

#         data = {
#             "Date": [current_date],
#             "Index": [index_data['indexName']],
#             "Current Value": [index_data['lastPrice']],
#             "Change": [index_data['change']],
#             "Change (%)": [index_data['pChange']]
#         }

#         df = pd.DataFrame(data)
#         file_name = "NIFTY_50_Live_Data.csv"
#         df.to_csv(file_name, index=False, mode='a', header=not bool(pd.read_csv(file_name).shape[0]))
#         print(f"NIFTY 50 live data saved to {file_name}.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# fetch_nifty_50_live()


An error occurred: Expecting value: line 1 column 1 (char 0)


In [11]:
# !pip install nsetools

In [1]:
#we have different csv files for different years with nifty 50 data
#concatenating them together
import os
import pandas as pd

directory = "./datasets/"

csv_files = sorted([f for f in os.listdir(directory) if f.startswith("NIFTY") and f.endswith(".csv")])

dfs = []
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv("./datasets/nifty_merged.csv", index=False)

print(f"Successfully merged {len(csv_files)} files into 'nifty_merged.csv'")


Successfully merged 11 files into 'nifty_merged.csv'


In [38]:
import pandas as pd

nifty_df = pd.read_csv("datasets/nifty_merged.csv")  
sp500_df = pd.read_csv("datasets/s&p500.csv")  

nifty_df.columns = nifty_df.columns.str.strip()
sp500_df.columns = sp500_df.columns.str.strip()

merged_df = pd.merge(nifty_df, sp500_df, on="Date", how="inner")

merged_df.rename(columns={"Closed_x": "NIFTY", "Closed_y": "S&P500"}, inplace=True)

merged_df.to_csv("stock_price_data.csv", index=False)

print("Successfully merged files into 'stock_price_data.csv'")


✅ Successfully merged files into 'stock_price_data.csv'


In [40]:
import pandas as pd

india_df = pd.read_csv("datasets/inflation_rates_india.csv")
us_df = pd.read_csv("datasets/inflation_rates_us.csv")

india_df_long = india_df.melt(id_vars=["Year"], var_name="Month", value_name="Inflation_Rate_India")
us_df_long = us_df.melt(id_vars=["Year"], var_name="Month", value_name="Inflation_Rate_US")

merged_df = pd.merge(india_df_long, us_df_long, on=["Year", "Month"], how="outer")

month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}
merged_df["Month"] = merged_df["Month"].map(month_map)

merged_df["Date"] = merged_df["Year"].astype(str) + "-" + merged_df["Month"] + "-01"

final_df = merged_df[["Date", "Inflation_Rate_India", "Inflation_Rate_US"]]

final_df = final_df.sort_values("Date").reset_index(drop=True)

final_df.to_csv("inflation_rates.csv", index=False)

print("Merged data saved to 'inflation_rates.csv'")


Merged data saved to 'inflation_rates.csv'


In [53]:
from fredapi import Fred
import pandas as pd

fred = Fred(api_key='aadcef49fb39a772a54e92142ac7f12c') #fred API Key

us_unemployment = fred.get_series('UNRATE')

us_unemployment_df = us_unemployment.to_frame(name='US_Unemployment_Rate')
us_unemployment_df.index.name = 'Date'
us_unemployment_df.reset_index(inplace=True)

us_unemployment_df['Date'] = pd.to_datetime(us_unemployment_df['Date'])
filtered_data = us_unemployment_df[(us_unemployment_df['Date'] >= '2015-01-01') & (us_unemployment_df['Date'] <= '2025-03-01')]

print(filtered_data.head())

filtered_data.to_csv('datasets/US_Unemployment_Rate.csv', index=False)


          Date  US_Unemployment_Rate
804 2015-01-01                   5.7
805 2015-02-01                   5.5
806 2015-03-01                   5.4
807 2015-04-01                   5.4
808 2015-05-01                   5.6


In [54]:
import pandas as pd

file1 = "datasets/India_Unemployment_Rate.csv"  
file2 = "datasets/US_Unemployment_Rate.csv"  

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

merged_df = pd.merge(df1, df2, on='Date', how='outer') 

output_file = "unemployment_rate.csv"
merged_df.to_csv(output_file, index=False)

print(f"Merged file saved as {output_file}")

Merged file saved as unemployment_rate.csv
